In [3]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
data = pd.read_csv('../data/story_point/bamboo.csv')
data = data[:10]
data['text'] = data['title'] + ' ' + data['description']
data = data[['text', 'storypoint']]

# 数据划分（80% 训练，20% 验证）
train_size = int(0.8 * len(data))
train_df = data[:train_size]
eval_df = data[train_size:]

# 转换为 Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

model_name = "Qwen/Qwen2-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1).to(torch.float32)

# 配置 LoRA 低秩适配
lora_config = LoraConfig(
    r=8,  # 低秩参数（较小值减少计算量）
    lora_alpha=32,
    target_modules=["query_key_value"],  # 适配注意力层
    lora_dropout=0.05,
    task_type=TaskType.SEQ_CLS  # 序列分类任务（适用于回归）
)

# 应用 LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # 仅微调少量参数，减少计算

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length")

train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# 重命名 'storypoint' 列为 'label'
train_dataset = train_dataset.rename_column("storypoint", "labels")
eval_dataset = eval_dataset.rename_column("storypoint", "labels")

# 保持数据格式
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

training_args = TrainingArguments(
    output_dir="./qwen2_lora_regression",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=2,  # LoRA 允许 batch_size 稍大
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    report_to="none",
    gradient_accumulation_steps=8,  # 累计梯度，相当于 batch_size 变大
    save_total_limit=2,
    dataloader_num_workers=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

trainer.train()

In [ ]:
trainer.evaluate()